In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [1]:
import pandas as pd
import sys
import os
sys.path.insert(0, '../../../')

from notebooks.utils import _ALEXA_DATA_PATH, load_node_features, load_level_data, create_audience_overlap_nodes, export_model_as_feature
from train import run_experiment
audience_overlap_sites = load_level_data(os.path.join(_ALEXA_DATA_PATH, 'corpus_2020_audience_overlap_sites_scrapping_result.json'), level=1)
audience_overlap_sites_NODES = create_audience_overlap_nodes(audience_overlap_sites)

#print(audience_overlap_sites_NODES[:5])
edge_df = pd.DataFrame(audience_overlap_sites_NODES, columns=['source', 'target'])
#edge_df.head()
#edge_df.shape
edge_df2 = pd.DataFrame()
edge_df2['source'] = edge_df['target']
edge_df2['target'] = edge_df['source']

edge_df = pd.concat([edge_df, edge_df2]).drop_duplicates(keep = "first").reset_index()
#edge_df.shape
nodes_in_edges = list(set(edge_df.source.unique().tolist() + edge_df.target.unique().tolist()))
#print('Number of unique nodes in edges:', len(nodes_in_edges), 'Sample:', nodes_in_edges[:5])
node_features_df = load_node_features()
node_features_df = node_features_df.set_index('site')
#node_features_df.head()
node_features_df = node_features_df.loc[nodes_in_edges]
#node_features_df.info()
node_features_df.alexa_rank = node_features_df.alexa_rank.fillna(1000000)
node_features_df.total_sites_linking_in = node_features_df.total_sites_linking_in.fillna(0)
node_features_df.daily_pageviews_per_visitor  = node_features_df.daily_pageviews_per_visitor.fillna(0)
node_features_df.daily_time_on_site = node_features_df.daily_time_on_site.fillna(0)
node_features_df.bounce_rate = node_features_df.bounce_rate.fillna(0)
#node_features_df.info()
import math

node_features_df['normalized_alexa_rank'] = node_features_df['alexa_rank'].apply(lambda x: 1/x if x else 0)
node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_in'].apply(lambda x: math.log2(x) if x else 0)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))

node_features_df[['alexa_rank', 'daily_pageviews_per_visitor', 'daily_time_on_site',
       'total_sites_linking_in', 'bounce_rate', 'normalized_alexa_rank',
       'normalized_total_sites_linked_in']] = scaler.fit_transform(node_features_df[['alexa_rank', 'daily_pageviews_per_visitor', 'daily_time_on_site',
       'total_sites_linking_in', 'bounce_rate', 'normalized_alexa_rank',
       'normalized_total_sites_linked_in']])
#node_features_df.head(5)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
2023-03-09 23:39:24.576898: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 23:39:28.916909: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 23:39:28.926387: I tensorflow/core/commo

In [2]:
labeled_nodes_df = pd.read_csv('corpus2020_node.csv')
# source_url_normalized, fact
labels_dict = {node_id: label for node_id, label in zip(labeled_nodes_df["source_url_normalized"], labeled_nodes_df["fact"])}
node_features_df
import numpy as np
train_mask = np.zeros(len(node_features_df), dtype=np.bool)
for node_id in labels_dict:
    if node_id in node_features_df.index:
        row_index = node_features_df.index.get_loc(node_id)
        train_mask[row_index] = True

node_features_df["train_mask"] = train_mask

node_features_df["label"] = node_features_df.index.map(labels_dict)
node_features_df.label = node_features_df.label.fillna(-99)
node_features_df.label, _ = pd.factorize(node_features_df.label, sort=True)

In [4]:
node_features_df

,alexa_rank,daily_pageviews_per_visitor,daily_time_on_site,total_sites_linking_in,bounce_rate,normalized_alexa_rank,normalized_total_sites_linked_in,train_mask,label
site,,,,,,,,,
newyorker.com,0.000225,0.040556,0.029066,1.208761e-02,0.794,4.107540e-04,0.705704,True,1
ferinheighttocelsius.com,0.254673,0.027778,0.000000,1.157963e-05,0.000,2.704561e-07,0.242444,False,0
thepolicy.us,0.092413,0.000000,0.000000,9.141815e-07,0.000,9.075871e-07,0.073222,False,0
lormariesplace.com,0.092413,0.000000,0.000000,2.742545e-06,0.000,9.075871e-07,0.146444,False,0
zcomm.org,0.019774,0.038889,0.013149,7.465816e-05,0.676,4.580943e-06,0.366656,True,1
...,...,...,...,...,...,...,...,...,...
themennonite.org,0.088885,0.027778,0.011995,6.064071e-05,0.875,9.472762e-07,0.352796,False,0
ociac.org,0.092413,0.000000,0.000000,1.828363e-06,0.000,9.075871e-07,0.119420,False,0
opptrends.com,0.005993,0.077778,0.041984,4.692798e-05,0.353,1.532874e-05,0.335711,False,0


In [3]:
node_map = {dom:i for i, dom in enumerate(node_features_df.index)}
edge_df3 = pd.DataFrame()
edge_df3['source'] = edge_df['source'].map(node_map)    
edge_df3['target'] = edge_df['target'].map(node_map)
edge_df3.head(5)

,source,target
0,9369,2345
1,9369,3967
2,9369,8838
3,9369,8393
4,9369,7483


In [6]:
import os.path as osp
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv, GraphConv, GravNetConv, ResGatedGraphConv
from torch_geometric.nn import SGConv, GATConv
from torch_geometric.loader import DataLoader
from torch_geometric.loader import NeighborLoader
from torch.utils.data import Dataset
from torch_geometric.nn import GNNExplainer
import pandas as pd
from torch_geometric.data import Data
from torch_geometric.loader import NeighborSampler #as RawNeighborSampler
import torch.nn as nn
import torch
from torch_cluster import random_walk
import torch.nn.functional as F
import numpy as np

def create_dataset(edges, features, labels, train_mask, test_mask):
    edge_index = torch.tensor(edges, dtype=torch.long)   
    # labels = np.array(labels)
    # labels = labels[~np.isnan(labels)]
    x = torch.tensor(features, dtype = torch.float)
    y = torch.tensor(labels, dtype = torch.long)
    train_mask = torch.tensor(train_mask, dtype = torch.bool)
    data = Data(x=x, edge_index=edge_index.t().contiguous(), train_mask = train_mask, y=y)
    return data

data = create_dataset(list(zip(edge_df3['source'], edge_df3['target'])),
                     list(zip(node_features_df['alexa_rank'], 
                              node_features_df['daily_pageviews_per_visitor'], 
                              node_features_df['daily_time_on_site'],
                              node_features_df['total_sites_linking_in'], 
                              node_features_df['bounce_rate'], 
                              node_features_df['normalized_alexa_rank'],
                              node_features_df['normalized_total_sites_linked_in'])),
                     node_features_df['label'], node_features_df['train_mask'], None)

In [7]:
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers, num_classes):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        
        for i in range(num_layers):
            in_channels = in_channels if i == 0 else hidden_channels
            self.convs.append(GCNConv(in_channels, hidden_channels))
        # post-message-passing
        self.post_mp = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels), nn.Dropout(0.25), 
            nn.Linear(hidden_channels, num_classes))

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.5, training=self.training)
        #return x
        x = self.post_mp(x)
        return F.log_softmax(x, dim=1)

In [8]:
def prepare(data, hidden_channels = 64, num_layers = 2, batch_size = 128):
    train_loader = NeighborLoader(data, input_nodes=None, num_neighbors=[10],
                            shuffle=True,  batch_size = 128)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("#features = {}".format(data.num_node_features))
    #print("Num layers ", num_layers)
    model = GCN(data.num_node_features, hidden_channels=hidden_channels, num_layers=num_layers, num_classes=3)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    x, edge_index = data.x.to(device), data.edge_index.to(device)
    return model, optimizer, x, edge_index, train_loader, device

In [13]:
def train(model, optimizer, train_loader, device):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        #train_mask = batch.train_mask
        out = model(batch.x.to(device), batch.edge_index.to(device))
        loss_labeled = F.cross_entropy(out[batch.train_mask].to(device), batch.y[batch.train_mask].to(device))
        loss_unlabeled = torch.tensor([0.], requires_grad=True) # Initialize loss for unlabeled nodes to zero
        for i in range(batch.num_nodes):
            if not batch.train_mask[i]: # If the node is unlabeled
                # Get the predicted label distribution for the node
                pred = out[i]
                # Compute the entropy of the distribution
                entropy = -torch.sum(torch.softmax(pred, dim=-1) * torch.log_softmax(pred, dim=-1))
                # Add the entropy to the total unlabeled loss
                loss_unlabeled = loss_unlabeled.to(device)
                loss_unlabeled = torch.add(loss_unlabeled, entropy)
            loss_labeled = loss_labeled.to(device)
            loss = loss_labeled + loss_unlabeled
        loss  = loss_labeled
        loss.backward()
        optimizer.step()
        total_loss += float(loss)
    return total_loss / data.num_nodes

In [15]:

def train_all(data, nepochs):
    model, optimizer, x, edge_index, train_loader, device = prepare(data)
    print(model)
    for epoch in range(1, nepochs+1):
        loss = train(model, optimizer, train_loader, device)
        #loss = train(model, data, optimizer, device)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    return model, x, edge_index

def predict_all(model, x, edge_index):
    with torch.no_grad():
        model.eval()
        print(edge_index.shape)
        out = model.full_forward(x, edge_index).cpu()
    return out

In [ ]:
model, x, edge_index = train_all(data, 1)